In [3]:
import ssl
import urllib.request
import torch
%load_ext autoreload

# Set autoreload to automatically reload all modules before executing code
%autoreload 2

# Create an SSL context that does not verify the certificate
ssl._create_default_https_context = ssl._create_unverified_context
from src.anti_spoof.fas import flip_it

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
model = flip_it()
model_path = "pretrained_models/msu_flip_mcl.pth.tar"
checkpoint = torch.load(model_path, map_location="cpu")
model.load_state_dict(checkpoint["state_dict"], strict=False)
model.eval()


cpu


flip_it(
  (model): CLIP(
    (visual): VisionTransformer(
      (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
      (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): Sequential(
          (0): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
            )
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=768, out_features=3072, bias=True)
              (gelu): QuickGELU()
              (c_proj): Linear(in_features=3072, out_features=768, bias=True)
            )
            (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          )
          (1): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLin